In [190]:
#!/Users/mahmoudshepero/anaconda3/bin/python3

import python as SP
from osgeo import ogr, osr
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import extractFiles as ex
import cars as cc

In [1]:
# load the buildings layer "LänmaterietLayer"
InputFileLocation = r'../Data/NewFolder/buildings-epsg3006.shp' # location
shapefile1 = InputFileLocation
buildingDS1 = ogr.Open(shapefile1)
buildingDS2 = ogr.Open(shapefile1)
buildingDS3 = ogr.Open(shapefile1)


# print the number of layers
print("Layer names:", SP.list_of_layers(buildingDS1))


# load the first layer
buildingsLayer = buildingDS1.GetLayer()

# check that the spatial reference is a metric spatial reference
print("spatial reference system: ",buildingsLayer.GetSpatialRef().ExportToWkt())

# filter to remove buildings with area less than 10m2
buildingsLayer.SetAttributeFilter("OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10")
buildingsLayer.ResetReading() # reset counting of the filter

# print the layer fields
print("Layer Fields:", SP.get_layer_fields(buildingsLayer))

# print the tags from the interesting fieldDefn
print("Field tags: ", SP.get_field_tags(buildingsLayer, "ANDAMAL_1"))


## RESIDENTIAL LAYER
# load the layer to residential
ResbuildingsLayer = buildingDS1.GetLayer()
# we are interested in the layer residential codes 133 and 135, 199
ResbuildingsLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (133, 135, 199)"
)
ResbuildingsLayer.ResetReading() # reset counting of the filter
print("Number of residential features: ", ResbuildingsLayer.GetFeatureCount())


## WORKPLACE LAYER
# load the layer to workplace
workplacesLayer = buildingDS2.GetLayer()
# workplaces are 240:299, 302, 304, 307, 311, 319, 322, 499
workplacesLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (240, 242, 243, 247, 248, 249, 252, 253, 299, "
"302, 304, 307, 311, 319, 322, 499)"
)
workplacesLayer.ResetReading() # reset counting of the filter
print("Number of workplace features: ", workplacesLayer.GetFeatureCount())


## OTHERS LAYER
# load the layer to other
otherLayer = buildingDS3.GetLayer()
# Otherplaces (leisure and shopping) are 301, 309, 313, 316, 317, 320, 399, 799
otherLayer.SetAttributeFilter( "OGR_GEOM_WKT LIKE 'POLYGON%' AND OGR_GEOM_AREA > 10"
"AND ANDAMAL_1 IN (301, 309, 313, 316, 317, 320, 399, 799)"
)
otherLayer.ResetReading() # reset counting of the filter
print("Number of other features: ", otherLayer.GetFeatureCount())


# Save the layers
SP.create_buffer_and_projectLayer(ResbuildingsLayer, 3006, 3006, "residentialLayer", bufferDist = 0)
SP.create_buffer_and_projectLayer(workplacesLayer, 3006, 3006, "workLayer", bufferDist = 0)
SP.create_buffer_and_projectLayer(otherLayer, 3006, 3006, "otherLayer", bufferDist = 0)




Layer names: ['buildings-epsg3006']
spatial reference system:  PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]
Layer Fields: ['DETALJTYP', 'ANDAMAL_1', 'ANDAMAL_1T', 'area']
Field tags:  {130, 131, 132, 133, 135, 399, 799, 299, 301, 302, 304, 305, 306, 307, 308, 309, 310, 313, 314, 699, 316, 317, 318, 319, 320, 321, 322, 315, 199, 599, 240, 242, 499, 243, 246, 247, 249, 252, 253}
Number of residential features:  5406
Number of workplace features:  2697
Number of other features:  1391


In [4]:
# Open Parking layer
InputFileLocation = r'../Data/NewFolder/OSM-epsg3600.shp' # location
shapefile2 = InputFileLocation
parkingDB = ogr.Open(shapefile2)
parkingLayer = parkingDB.GetLayer()

# print the parking layer tags
print("Amenity Field tags: ", SP.get_field_tags(parkingLayer, "amenity"))
print("Building Field tags: ", SP.get_field_tags(parkingLayer, "building"))
# Remove None features
parkingLayer.SetAttributeFilter( "NOT OGR_GEOM_WKT LIKE 'None%' AND OGR_GEOM_AREA > 10"
  "AND (amenity IN ('parking', 'parking_space') OR building in ('garage', 'garages'))")
parkingLayer.ResetReading() # reset counting of the filter
print("Number of parking features: ", parkingLayer.GetFeatureCount())


# Open Buffered layer
InputFileLocation = r'UppsalaParkingBuffer100meter3006.shp' # location
shapefile3 = InputFileLocation
parkingBuffer = ogr.Open(shapefile3)
parkingBufferLayer = parkingBuffer.GetLayer()


Field tags:  {'kindergarten', 'university', 'place_of_worship', 'shelter', 'car_rental', 'social_facility', 'parking', 'doctors', 'fast_food', 'theatre', 'recycling', 'social_centre', 'fountain', 'events_venue', 'restaurant', 'nursing_home', 'boat_storage', 'concert_hall', 'cafe', 'community_centre', 'vehicle_inspection', 'childcare', 'nightclub', 'animal_shelter', 'school', 'animal_training', 'waste_disposal', 'car_wash', 'toilets', 'monastery', 'clinic', 'fire_station', 'library', 'prison', 'parking_space', None, 'bar', 'hospital', 'bicycle_parking', 'motorcycle_parking', 'grave_yard', 'fuel'}
Field tags:  {'public', 'kindergarten', 'university', 'apartments', 'industrial', 'hospital', 'train_station', 'detached', 'terrace', 'service', 'hut', 'house', 'office', 'warehouse', 'farm_auxiliary', 'garage', 'boathouse', 'shed', 'commercial', 'hotel', 'sport', 'school', 'church', 'farm', 'silo', 'retail', 'cabin', 'hangar', 'roof', 'residential', None, 'yes', 'garages', 'greenhouse', 'barn'

In [ ]:
# Save the figure
fig = plt.figure(figsize = (500,500))
SP.plot_features(parkingLayer, "parkingPlot.pdf", '#555577')
SP.plot_features(otherLayer, "parkingPlot.pdf", '#118877')
SP.plot_features(workplacesLayer, "parkingPlot.pdf", '#992266')
SP.plot_features(ResbuildingsLayer, "parkingPlot.pdf", '#228811')
fig.savefig("parkingPlot1.pdf")


In [46]:
# Calculate intersection of the buffered parking lot layer with the buidlings layers
areaPercentage = SP.get_percentage_of_area_types(parkingBufferLayer,
                [ResbuildingsLayer, workplacesLayer, otherLayer])
np.savetxt("areaPercentage.txt", areaPercentage)

# Get the areas of the parking lots
parkingLotAreas = SP.get_features_areas(parkingLayer)

In [43]:
# Get station IDs from the data set
ID = SP.get_field_tags(parkingLayer, "osm_way_id")
# Assert there are no duplicate IDs
assert len(ID) == parkingLayer.GetFeatureCount(), "there are duplicate IDs, create your own IDs for the stations" 


In [166]:
stations = SP.create_charging_stations(ID, 
                               parkingLotAreas, 
                               areaPercentage,
                               areaPerCar = 15,     
                               charging_status = True, 
                               charging_power = 3.7)

In [167]:
# Number of state 0 stations
print("number of home stations:", 
      len(list(iter.filterfalse(lambda x: not(x.state ==0),stations))))
# Number of state 1 stations
print("number of work stations:", 
      len(list(iter.filterfalse(lambda x: not(x.state ==1),stations))))
# Number of state 2 stations
print("number of other stations:", 
      len(list(iter.filterfalse(lambda x: not(x.state ==2),stations))))


number of home stations: 1350
number of work stations: 979
number of other stations: 733


In [168]:
# Number of parking palaces for state 0 stations
print("number of home stations:", 
      sum(list(map(lambda x: x.maximumOccupancy, list(iter.filterfalse(lambda x: not(x.state ==0),stations))))))
# Number of parking palaces for state 1 stations
print("number of work stations:", 
      sum(list(map(lambda x: x.maximumOccupancy, list(iter.filterfalse(lambda x: not(x.state ==1),stations))))))
# Number of parking palaces for state 2 stations
print("number of other stations:", 
      sum(list(map(lambda x: x.maximumOccupancy, list(iter.filterfalse(lambda x: not(x.state ==2),stations))))))



number of home stations: 36639
number of work stations: 50552
number of other stations: 15982


In [ ]:
# Create EVs
numberCars = 100

cars = [EV(currentLocation = 1, currentState = 0, batteryCharge = 0.0,
                 batteryCapacity = 0.0, trips = 0) for i in range(numberCars)]


In [191]:
# load transition Matrix
weekdayChain = ex.readMatrixfiles("../TransitionMatrix/*weekday*.txt")
# define the tranistion Matrix
weekday = cc.Markov(weekdayChain)

In [ ]:
# Setup the simulation Model
simulationCase = cc.Simulation(stations,
                            cars,
                            weekday)

# Estimate the electric load
load = simulationCase.simulate_model(10080)

In [165]:
import imp; imp.reload(SP)

<module 'python' from '/Users/mahmoudshepero/Documents/PhD/Models/EVSpatialModelPython/PythonCode/python.py'>